# Exploring Genomic Data

This notebook demonstrates working with genetic variant data stored as publicly accessible Google BigQuery datasets.

Specifically, we will work with the [Illumina Platinum Genomes](https://cloud.google.com/genomics/data/platinum-genomes) data. The source data was originally in [VCF format](http://vcftools.sourceforge.net/VCF-poster.pdf), which was imported to [Google Genomics](https://cloud.google.com/genomics/) and exported to BigQuery.

If you want to explore other genomics samples, see https://github.com/googlegenomics/datalab-examples. You can import them into your Google Cloud Datalab instance by uploading them from the notebook list page.

In [1]:
import google.datalab.bigquery as bq

## Variants

Let's start with a little bit of genomics nomenclature:

 * **Variant**: A region of the genome that has been identified as differing from the reference genome.
 * **Non-variant Segment**: A region of the genome that matches the reference genome. 
 * **Reference Name**: The name of a reference segment of DNA. Typically, this is a chromosome, but it may be other named regions from a reference genome.
 * **Reference Bases**: The A's, C's, T's, and G's comprising the DNA of the reference genome at the site of variation.
 * **Alternate Bases**: The A's, C's, T's, and G's comprising the DNA actually found at the site of variation for the sample(s) sequenced.
 * **SNP**: A [single nucleotide polymorphism](https://en.wikipedia.org/wiki/Single-nucleotide_polymorphism) is a DNA sequence variation in which a single nucleotide — A, T, C or G — in the genome (or other shared sequence) differs between members of a biological species or paired chromosomes.


In [2]:
variants = bq.Table('genomics-public-data.platinum_genomes.variants')
variants.length

261285806

Wow, that's a lot of records!  For a detailed description of the schema, see [Understanding the BigQuery Variants Table Schema](https://cloud.google.com/genomics/v1/bigquery-variants-schema).

## SNPs

Next, let's take a look at a few SNPs in our data.  As mentioned in the nomenclature, SNPs are a particular kind of genetic variant. Let's create an SQL statement that can list all variant records for a single base change:

In [3]:
%load_ext google.datalab.kernel

In [4]:
%%bq query --name single_base
SELECT
  reference_name,
  start,
  reference_bases,
  alternate_bases
FROM
  `genomics-public-data.platinum_genomes.variants`
WHERE
  reference_bases IN ('A','C','G','T') AND
  ARRAY_LENGTH(alternate_bases) = 1
LIMIT 100

In [5]:
%bq execute --query single_base

reference_name,start,reference_bases,alternate_bases
chr14,44018132,C,['G']
chr8,4515415,A,['T']
chr7,158995897,C,['G']
chr5,178600223,A,['C']
chr14,29080321,G,['A']
chr10,20971204,G,['A']
chr2,53991706,A,['C']
chr13,19663322,C,['T']
chr21,15029525,G,['T']
chr13,72009601,A,['G']


## Transition/Transversion Ratio

A SNP can be further classified as either a [transition](https://en.wikipedia.org/wiki/Transition_%28genetics%29) or a [transversion](https://en.wikipedia.org/wiki/Transversion). The ratio of transitions to transversions (TiTv ratio) in humans is observed to be approximately 2.1, but this is not uniform across the genome. Let's take a closer look by computing the TiTv ratio in contiguous regions of 100,000 base pairs.

In [6]:
%%bq query --name snps
SELECT
  reference_name,
  reference_bases,
  alternate_bases,
  CAST(FLOOR(start / 100000) AS INT64) AS windows,
  CONCAT(reference_bases, CONCAT(CAST('->' AS STRING), ARRAY_TO_STRING(alternate_bases, ''))) AS mutation,
  ARRAY_LENGTH(alternate_bases) AS num_alts
FROM
  `genomics-public-data.platinum_genomes.variants`
WHERE reference_name IN ("1", "chr1")
  # Limit to bi-allelic SNP variants
  AND reference_bases IN ('A','C','G','T')
  AND ARRAY_LENGTH(alternate_bases) = 1

We've updated the above query to include the "window" in which the SNP resides, and added a new field called "mutation".

In [7]:
%bq sample -q snps --count 10

reference_name,reference_bases,alternate_bases,windows,mutation,num_alts
chr1,G,['A'],1497,G->A,1
chr1,G,['C'],695,G->C,1
chr1,A,['G'],1545,A->G,1
chr1,G,['A'],2158,G->A,1
chr1,C,['T'],599,C->T,1
chr1,T,['G'],545,T->G,1
chr1,C,['T'],468,C->T,1
chr1,G,['T'],2363,G->T,1
chr1,A,['G'],1475,A->G,1
chr1,G,['A'],1899,G->A,1


Next we group and classify the SNPs within their windows.

In [8]:
%%bq query --name windows --subqueries snps
SELECT
  reference_name,
  windows AS win,
  COUNTIF(mutation IN ('A->G', 'G->A', 'C->T', 'T->C')) AS transitions,
  COUNTIF(mutation IN ('A->C', 'C->A', 'G->T', 'T->G',
                       'A->T', 'T->A', 'C->G', 'G->C')) AS transversions,
  COUNT(mutation) AS num_variants_in_window
  FROM snps
GROUP BY
  reference_name,
  win

In [9]:
%bq sample -q windows --count 10

reference_name,win,transitions,transversions,num_variants_in_window
chr1,986,202,136,365
chr1,545,110,74,198
chr1,878,186,83,280
chr1,140,109,103,224
chr1,2436,145,110,265
chr1,1538,187,157,378
chr1,1741,141,114,277
chr1,233,142,112,270
chr1,1556,92,88,194
chr1,2492,152,124,283


And finally, we compute the per-window TiTv ratio.

In [10]:
%%bq query --name titv --subqueries snps windows
SELECT
  reference_name,
  win * 100000 AS window_start,
  transitions,
  transversions,
  transitions/transversions AS titv,
  num_variants_in_window
FROM windows
ORDER BY
    window_start

In [11]:
%bq sample -q titv --count 10

reference_name,window_start,transitions,transversions,titv,num_variants_in_window
chr1,0,232,156,1.4871794871794872,397
chr1,100000,118,56,2.107142857142857,177
chr1,200000,55,37,1.4864864864864864,94
chr1,300000,1,8,0.125,9
chr1,400000,21,11,1.9090909090909092,32
chr1,500000,160,49,3.2653061224489797,210
chr1,600000,63,43,1.4651162790697674,107
chr1,700000,278,252,1.1031746031746033,544
chr1,800000,906,619,1.4636510500807753,1562
chr1,900000,365,300,1.2166666666666666,698


In [12]:
titvRatios = titv.execute(output_options=bq.QueryOutput.dataframe()).result()
titvRatios[:5]

,reference_name,window_start,transitions,transversions,titv,num_variants_in_window
0,chr1,0,232,156,1.487179,397
1,chr1,100000,118,56,2.107143,177
2,chr1,200000,55,37,1.486486,94
3,chr1,300000,1,8,0.125000,9
4,chr1,400000,21,11,1.909091,32


## Visualization

Now we can take the ratios and plot them by genomic position to see how the ratio varies depending upon where it was computed within the chromosome. By default, this plot shows chromosome 1 with its gap in the center of the data corresponding to its [metacentric contromere](https://en.wikipedia.org/wiki/Centromere#Metacentric).

In [13]:
import seaborn as sns
import matplotlib.pyplot as plt

g = sns.lmplot(x='window_start',
               y='titv',
               data = titvRatios,
               size = 8,
               aspect = 2,
               scatter_kws = { 'color': 'black', 'alpha': 0.4 },
               line_kws = { 'color': 'blue', 'alpha': 0.5, 'lw': 4 },
               lowess = True)
plt.xlabel('Genomic Position', fontsize = 14)  
plt.ylabel('Ti/Tv Ratio', fontsize = 14)
plt.title('Ti/Tv Ratio computed on %d base pair windows for %s' % (100000, ['1', 'chr1']),
          fontsize = 18)
plt.annotate('Centromere', xy = (1.3e8, 1.5), xytext = (1.5e8, 3), size = 14,
             arrowprops=dict(facecolor='black', shrink=0.05))

/Users/ajhamilton/notebooks/venv/lib/python3.6/site-packages/seaborn/regression.py:546: UserWarning: The `size` paramter has been renamed to `height`; please update your code.
  warnings.warn(msg, UserWarning)


ModuleNotFoundError: No module named 'statsmodels'